In [ ]:
!pip install gcloud


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602926 sha256=48be2f7a06256aa234d7a54961cbb8a823ffc580f996f013be9ac41a2821696c
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [ ]:
!gcloud auth application-default login


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=kPuewgRZg9oarrgN7VxibgLftIZg7N&prompt=consent&token_usage=remote&access_type=offline&code_challenge=KOnBU0zAx-0kW-duG3SRQIruh8anG_sfntGWARRuYAY&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrCbK-H_dIsXJd0S4g-OBBg0RSERXvFw4IyVDAo1RmgNQ5mWw_thfAtd9bqTr88Qyg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [ ]:
client = bigquery.Client(project='repositoriodedadosgpsp')
query = "SELECT * FROM `repositoriodedadosgpsp.Datalake.STN_qualidade_fiscal_uf` "
df = client.query(query).to_dataframe()
df

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,ano,sigla_uf,id_uf,indicador_qualidade_informacao_contabil_fiscal,posicao_ranking,quantidade_acertos_total,proporcao_acertos,dimensao_I,dimensao_II,dimensao_III,dimensao_IV
0,2022,SE,28,A,10,126.517,0.951,32.517,55.000,12,27
1,2020,RO,11,A,3,91.380,0.951,19.380,40.000,13,19
2,2020,MS,50,A,2,91.470,0.952,19.470,40.000,13,19
3,2021,AM,13,A,3,96.759,0.968,22.759,42.000,13,19
4,2021,MS,50,A,8,95.661,0.957,22.661,41.000,13,19
...,...,...,...,...,...,...,...,...,...,...,...
130,2020,RR,14,D,26,66.910,0.696,15.910,32.000,11,8
131,2020,MA,21,D,25,71.450,0.744,18.450,36.000,11,6
132,2019,RR,14,E,27,8.300,0.122,8.300,0.000,0,0
133,2023,AP,16,E,27,95.632,0.625,30.831,44.801,8,12


In [ ]:
df['ano'].unique()

<IntegerArray>
[2022, 2020, 2021, 2023, 2019]
Length: 5, dtype: Int64

In [ ]:
#Alterando tipos para manter fidelidade ao schema
df['id_uf'] = df['id_uf'].astype(str)
df['dimensao_II'] = df['dimensao_II'].astype(int)

In [ ]:
df

,ano,sigla_uf,id_uf,indicador_qualidade_informacao_contabil_fiscal,posicao_ranking,quantidade_acertos_total,proporcao_acertos,dimensao_I,dimensao_II,dimensao_III,dimensao_IV
0,2022,SE,28,A,10,126.517,0.951,32.517,55,12,27
1,2020,RO,11,A,3,91.380,0.951,19.380,40,13,19
2,2020,MS,50,A,2,91.470,0.952,19.470,40,13,19
3,2021,AM,13,A,3,96.759,0.968,22.759,42,13,19
4,2021,MS,50,A,8,95.661,0.957,22.661,41,13,19
...,...,...,...,...,...,...,...,...,...,...,...
130,2020,RR,14,D,26,66.910,0.696,15.910,32,11,8
131,2020,MA,21,D,25,71.450,0.744,18.450,36,11,6
132,2019,RR,14,E,27,8.300,0.122,8.300,0,0,0
133,2023,AP,16,E,27,95.632,0.625,30.831,44,8,12


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 11 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   ano                                             135 non-null    Int64  
 1   sigla_uf                                        135 non-null    object 
 2   id_uf                                           135 non-null    object 
 3   indicador_qualidade_informacao_contabil_fiscal  135 non-null    object 
 4   posicao_ranking                                 135 non-null    Int64  
 5   quantidade_acertos_total                        135 non-null    float64
 6   proporcao_acertos                               135 non-null    float64
 7   dimensao_I                                      135 non-null    float64
 8   dimensao_II                                     135 non-null    int64  
 9   dimensao_III                               

In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('id_uf','STRING',description='Código da Unidade da Federação.'),
 bigquery.SchemaField('indicador_qualidade_informacao_contabil_fiscal','STRING',description='Nota do indicador de qualidade'),
 bigquery.SchemaField('posicao_ranking','INTEGER',description='Posição do ranking.'),
 bigquery.SchemaField('quantidade_acertos_total','FLOAT',description='Quantidade de acertos em todas as dimensões'),
 bigquery.SchemaField('proporcao_acertos','FLOAT',description='Proporção de acertos'),
 bigquery.SchemaField('dimensao_I','FLOAT',description='Dimensão I - Gestão da Informação.'),
 bigquery.SchemaField('dimensao_II','INTEGER',description='Dimensão II - Informações Contábeis.'),
 bigquery.SchemaField('dimensao_III','INTEGER',description='Dimensão III - Informações Fiscais.'),
 bigquery.SchemaField('dimensao_IV','INTEGER',description='Dimensão IV - Informações Contábeis x Informações Fiscais')
 ]

In [ ]:
#Dimensão I - Gestão da Informação.
#Dimensão II - Informações Contábeis.
#Dimensão III - Informações Fiscais.
#Dimensão IV - Informações Contábeis x Informações Fiscais.

In [ ]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('participacao_transparencia')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
table_ref = dataset_ref.table('STN_qualidade_fiscal_uf_v2') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=df81f94f-6e21-470c-8e1c-9909b3653cf3>